In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt


from principal_DNN_MNIST import init_DNN, pretrain_DNN, retropropagation, test_DNN, entree_sortie_reseau
from utils import read_mnist, save_object, plot_error_rates, download_data, save_dict_to_json

In [ ]:
download_data()
X_train, X_test, y_train, y_test = read_mnist()

In [ ]:
batch_size=128 
n_iter_rbm=100
learning_rate=0.01 
n_iter_dbn=200
verbose=True

# Number of Layers

In [ ]:
layer_configurations = [
    (784, 200, 10),
    (784, 200, 200, 10),
    (784, 200, 200, 200, 10),
    (784, 200, 200, 200, 200, 10),
    (784, 200, 200, 200, 200, 200, 10)
]
# Store error rates
error_rates_pretrained = {}
error_rates_not_pretrained = {}

# Store losses
losses_finetune_not_pretrained_dict = {}
losses_pretrain_dict = {}
losses_finetune_pretrained_dict = {}

base_dir = "DNN_Models"

print("Starting experiments...")

if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    print(f"Created base directory {base_dir} for saving models.")

for index, neurons in enumerate(layer_configurations, start=1):
    print(f"\nExperiment with Layer Configuration {index}: {neurons}")
    
    config_dir = os.path.join(base_dir, '_'.join(map(str, neurons)))
    if not os.path.exists(config_dir):
        os.makedirs(config_dir)
        print(f"Created configuration directory: {config_dir}")

    error_rates_pretrained[neurons] = {'train': None, 'test': None}
    error_rates_not_pretrained[neurons] = {'train': None, 'test': None}

    print("Initializing DNNs...")
    dnn_pretrained = init_DNN(neurons)
    dnn_not_pretrained = init_DNN(neurons)

    print("Training non pretrained DNNs...")
    dnn_not_pretrained, losses_finetune_not_pretrained = retropropagation(dnn_not_pretrained, X_train, y_train, n_iter_dbn, learning_rate, batch_size)
    losses_finetune_not_pretrained_dict[neurons] = losses_finetune_not_pretrained

    print("Training completed. Evaluating fine-tuned models...")
    error_not_pretrained_train = test_DNN(dnn_not_pretrained, X_train, y_train, verbose=False)
    print(f'Error not Pretrained for train: {error_not_pretrained_train}')
    error_rates_not_pretrained[neurons]['train'] = error_not_pretrained_train
    
    error_not_pretrained_test = test_DNN(dnn_not_pretrained, X_test, y_test, verbose=False)
    print(f'Error not Pretrained for test: {error_not_pretrained_test}')
    error_rates_not_pretrained[neurons]['test'] = error_not_pretrained_test

    print("Saving fine-tuned models...")
    save_object(dnn_not_pretrained, os.path.join(config_dir, "dnn_not_pretrained_finetuned.pkl"))
    
    
    print("Pre-training one DNN...")
    dnn_pretrained, losses_pretrain = pretrain_DNN(
        dnn_pretrained, 
        X_train, 
        epochs=n_iter_rbm, 
        learning_rate=learning_rate, 
        batch_size=batch_size, 
        verbose=verbose
    )
    losses_pretrain_dict[neurons] = losses_pretrain
    print("Pre-training completed. Saving pre-trained model and losses...")
    save_object(dnn_pretrained, os.path.join(config_dir, "dnn_pretrained.pkl"))
    save_object(losses_pretrain, os.path.join(config_dir, "losses_pretrain.pkl"))
    
    print("Training pretrained DNNs...")
    dnn_pretrained, losses_finetune_pretrained = retropropagation(dnn_pretrained, X_train, y_train, n_iter_dbn, learning_rate, batch_size)
    losses_finetune_pretrained_dict[neurons] = losses_finetune_pretrained

    print("Training completed. Evaluating fine-tuned models...")
    error_pretrained_train = test_DNN(dnn_pretrained, X_train, y_train, verbose=False)
    print(f'Error Pretrained for train: {error_pretrained_train}')
    error_rates_pretrained[neurons]['train'] = error_pretrained_train

    error_pretrained_test = test_DNN(dnn_pretrained, X_test, y_test, verbose=False)
    print(f'Error Pretrained for test: {error_pretrained_test}')
    error_rates_pretrained[neurons]['test'] = error_pretrained_test

    print("Saving fine-tuned models...")
    save_object(dnn_pretrained, os.path.join(config_dir, "dnn_pretrained_finetuned.pkl"))

    # Convert the tuple keys to strings for JSON compatibility
    error_rates_pretrained_json = {str(k): v for k, v in error_rates_pretrained.items()}
    error_rates_not_pretrained_json = {str(k): v for k, v in error_rates_not_pretrained.items()}
    
    # Save the dictionaries as JSON in the specified base directory
    save_dict_to_json(error_rates_pretrained_json, os.path.join(base_dir, "error_rates_pretrained_n_layers.json"))
    save_dict_to_json(error_rates_not_pretrained_json, os.path.join(base_dir, "error_rates_not_pretrained_n_layers.json"))
    
    print("Both dictionaries were saved as JSON files in the DNN_Models directory.")

print("All experiments completed. Plotting results...")

plot_error_rates(
    error_rates_pretrained, 
    error_rates_not_pretrained, 
    [1, 2, 3, 4, 5],
    "Number of Layers", 
    "experiment_layers.png"
)

# Number of neurons by layer

In [ ]:
layer_configurations = [
    (784, 100, 100, 10),
    (784, 200, 200, 10),
    (784, 300, 300, 10),
    (784, 400, 400, 10),
    (784, 500, 500, 10),
    (784, 600, 600, 10),
    (784, 700, 700, 10),
]

# Store error rates
error_rates_pretrained = {}
error_rates_not_pretrained = {}

# Store losses
losses_finetune_not_pretrained_dict = {}
losses_pretrain_dict = {}
losses_finetune_pretrained_dict = {}

base_dir = "DNN_Models"

print("Starting experiments...")

if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    print(f"Created base directory {base_dir} for saving models.")

for index, neurons in enumerate(layer_configurations, start=1):
    print(f"\nExperiment with Layer Configuration {index}: {neurons}")
    
    config_dir = os.path.join(base_dir, '_'.join(map(str, neurons)))
    if not os.path.exists(config_dir):
        os.makedirs(config_dir)
        print(f"Created configuration directory: {config_dir}")

    error_rates_pretrained[neurons] = {'train': None, 'test': None}
    error_rates_not_pretrained[neurons] = {'train': None, 'test': None}

    print("Initializing DNNs...")
    dnn_pretrained = init_DNN(neurons)
    dnn_not_pretrained = init_DNN(neurons)


    print("Training non pretrained DNNs...")
    dnn_not_pretrained, losses_finetune_not_pretrained = retropropagation(dnn_not_pretrained, X_train, y_train, n_iter_dbn, learning_rate, batch_size)
    losses_finetune_not_pretrained_dict[neurons] = losses_finetune_not_pretrained

    print("Training completed. Evaluating fine-tuned models...")
    error_not_pretrained_train = test_DNN(dnn_not_pretrained, X_train, y_train, verbose=False)
    print(f'Error not Pretrained for train: {error_not_pretrained_train}')
    error_rates_not_pretrained[neurons]['train'] = error_not_pretrained_train
    
    error_not_pretrained_test = test_DNN(dnn_not_pretrained, X_test, y_test, verbose=False)
    print(f'Error not Pretrained for test: {error_not_pretrained_test}')
    error_rates_not_pretrained[neurons]['test'] = error_not_pretrained_test

    print("Saving fine-tuned models...")
    save_object(dnn_not_pretrained, os.path.join(config_dir, "dnn_not_pretrained_finetuned.pkl"))
    
    
    print("Pre-training one DNN...")
    dnn_pretrained, losses_pretrain = pretrain_DNN(
        dnn_pretrained, 
        X_train, 
        epochs=n_iter_rbm, 
        learning_rate=learning_rate, 
        batch_size=batch_size, 
        verbose=verbose
    )
    losses_pretrain_dict[neurons] = losses_pretrain
    print("Pre-training completed. Saving pre-trained model and losses...")
    save_object(dnn_pretrained, os.path.join(config_dir, "dnn_pretrained.pkl"))
    save_object(losses_pretrain, os.path.join(config_dir, "losses_pretrain.pkl"))
    
    print("Training pretrained DNNs...")
    dnn_pretrained, losses_finetune_pretrained = retropropagation(dnn_pretrained, X_train, y_train, n_iter_dbn, learning_rate, batch_size)
    losses_finetune_pretrained_dict[neurons] = losses_finetune_pretrained

    print("Training completed. Evaluating fine-tuned models...")
    error_pretrained_train = test_DNN(dnn_pretrained, X_train, y_train, verbose=False)
    print(f'Error Pretrained for train: {error_pretrained_train}')
    error_rates_pretrained[neurons]['train'] = error_pretrained_train

    error_pretrained_test = test_DNN(dnn_pretrained, X_test, y_test, verbose=False)
    print(f'Error Pretrained for test: {error_pretrained_test}')
    error_rates_pretrained[neurons]['test'] = error_pretrained_test

    print("Saving fine-tuned models...")
    save_object(dnn_pretrained, os.path.join(config_dir, "dnn_pretrained_finetuned.pkl"))

    # Convert the tuple keys to strings for JSON compatibility
    error_rates_pretrained_json = {str(k): v for k, v in error_rates_pretrained.items()}
    error_rates_not_pretrained_json = {str(k): v for k, v in error_rates_not_pretrained.items()}
    
    # Save the dictionaries as JSON in the specified base directory
    save_dict_to_json(error_rates_pretrained_json, os.path.join(base_dir, "error_rates_pretrained_n_neurons.json"))
    save_dict_to_json(error_rates_not_pretrained_json, os.path.join(base_dir, "error_rates_not_pretrained_n_neurons.json"))
    
    print("Both dictionaries were saved as JSON files in the DNN_Models directory.")

print("All experiments completed. Plotting results...")

plot_error_rates(
    error_rates_pretrained, 
    error_rates_not_pretrained, 
    [100, 200, 300, 400, 500, 600, 700],
    "Number of neurons in hidden layers", 
    "experiment_n_neurons.png"
)

# Number of samples 

In [ ]:
# The single layer configuration to use for this experiment
layer_configuration = (784, 200, 200, 10)

# The different sample sizes for the training set
sample_sizes = [1000, 3000, 7000, 10000, 30000, 60000]

# Store error rates
error_rates_pretrained = {}
error_rates_not_pretrained = {}

# Store losses
losses_finetune_not_pretrained_dict = {}
losses_pretrain_dict = {}
losses_finetune_pretrained_dict = {}

base_dir = "DNN_Models"

if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    print(f"Created base directory {base_dir} for saving models.")

print("Starting experiments...")

for sample_size in sample_sizes:
    print(f"\nExperiment with Sample Size: {sample_size}")

    # Shuffle the indices
    indices = np.random.permutation(len(X_train))
    
    # Select a subset of the training data and labels
    X_train_sub = X_train[indices[:sample_size]]
    y_train_sub = y_train[indices[:sample_size]]

    print("Subset of training data shape:", X_train_sub.shape, "Subset of training labels shape:", y_train_sub.shape)
    
    config_dir_name = '_'.join(map(str, layer_configuration)) + f"_samples_{sample_size}"
    config_dir = os.path.join(base_dir, config_dir_name)
    
    if not os.path.exists(config_dir):
        os.makedirs(config_dir)
        print(f"Created configuration directory: {config_dir}")
    
    error_rates_pretrained[sample_size] = {'train': None, 'test': None}
    error_rates_not_pretrained[sample_size] = {'train': None, 'test': None}

    print("Initializing DNNs...")
    dnn_pretrained = init_DNN(layer_configuration)
    dnn_not_pretrained = init_DNN(layer_configuration)


    print("Training non pretrained DNNs...")
    dnn_not_pretrained, losses_finetune_not_pretrained = retropropagation(dnn_not_pretrained, X_train_sub, y_train_sub, n_iter_dbn, learning_rate, batch_size)
    losses_finetune_not_pretrained_dict[sample_size] = losses_finetune_not_pretrained

    print("Training completed. Evaluating fine-tuned models...")
    error_not_pretrained_train = test_DNN(dnn_not_pretrained, X_train_sub, y_train_sub, verbose=False)
    print(f'Error not Pretrained for train: {error_not_pretrained_train}')
    error_rates_not_pretrained[sample_size]['train'] = error_not_pretrained_train
    
    error_not_pretrained_test = test_DNN(dnn_not_pretrained, X_test, y_test, verbose=False)
    print(f'Error not Pretrained for test: {error_not_pretrained_test}')
    error_rates_not_pretrained[sample_size]['test'] = error_not_pretrained_test

    print("Saving fine-tuned models...")
    save_object(dnn_not_pretrained, os.path.join(config_dir, "dnn_not_pretrained_finetuned.pkl"))
    
    
    print("Pre-training one DNN...")
    dnn_pretrained, losses_pretrain = pretrain_DNN(
        dnn_pretrained, 
        X_train_sub, 
        epochs=n_iter_rbm, 
        learning_rate=learning_rate, 
        batch_size=batch_size, 
        verbose=verbose
    )
    losses_pretrain_dict[sample_size] = losses_pretrain
    print("Pre-training completed. Saving pre-trained model and losses...")
    save_object(dnn_pretrained, os.path.join(config_dir, "dnn_pretrained.pkl"))
    save_object(losses_pretrain, os.path.join(config_dir, "losses_pretrain.pkl"))
    
    print("Training pretrained DNNs...")
    dnn_pretrained, losses_finetune_pretrained = retropropagation(dnn_pretrained, X_train_sub, y_train_sub, n_iter_dbn, learning_rate, batch_size)
    losses_finetune_pretrained_dict[sample_size] = losses_finetune_pretrained

    print("Training completed. Evaluating fine-tuned models...")
    error_pretrained_train = test_DNN(dnn_pretrained, X_train_sub, y_train_sub, verbose=False)
    print(f'Error Pretrained for train: {error_pretrained_train}')
    error_rates_pretrained[sample_size]['train'] = error_pretrained_train

    error_pretrained_test = test_DNN(dnn_pretrained, X_test, y_test, verbose=False)
    print(f'Error Pretrained for test: {error_pretrained_test}')
    error_rates_pretrained[sample_size]['test'] = error_pretrained_test

    print("Saving fine-tuned models...")
    save_object(dnn_pretrained, os.path.join(config_dir, "dnn_pretrained_finetuned.pkl"))

    # Convert the tuple keys to strings for JSON compatibility
    error_rates_pretrained_json = {str(k): v for k, v in error_rates_pretrained.items()}
    error_rates_not_pretrained_json = {str(k): v for k, v in error_rates_not_pretrained.items()}
    
    # Save the dictionaries as JSON in the specified base directory
    save_dict_to_json(error_rates_pretrained_json, os.path.join(base_dir, "error_rates_pretrained_n_samples.json"))
    save_dict_to_json(error_rates_not_pretrained_json, os.path.join(base_dir, "error_rates_not_pretrained_n_samples.json"))
    
    print("Both dictionaries were saved as JSON files in the DNN_Models directory.")

print("All experiments completed. Plotting results...")

plot_error_rates(
    error_rates_pretrained, 
    error_rates_not_pretrained, 
    sample_sizes,
    "Number of samples", 
    "experiment_samples.png"
)